In [ ]:
"""
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
pip freeze > requirements.txt


git clone https://github.com/kbandla/APTnotes.git
"""

In [1]:
from huggingface_hub import login

with open('huggingface_token.txt', 'r') as f:
    token = f.read()

login(token)

/Users/zpietrza/AI-Generated-CTI/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
from datasets import Dataset
data = {
    "input": ["Hello, how are you?", "What's your name?"],
    "output": ["I'm doing great!", "My name is Mistral."]
}

dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 2
})

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples['input'], padding="max_length", truncation=True)

# Tokenizacja danych
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 2
})

In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    save_steps=500,
    report_to="wandb",              # WandB training monitoring

    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Adjust to memory
    num_train_epochs=1,
    weight_decay=0.01,              # Regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # TODO
)

trainer.train()
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

RuntimeError: MPS backend out of memory (MPS allocated: 20.38 GB, other allocations: 464.00 KB, max allowed: 20.40 GB). Tried to allocate 224.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
model = AutoModelForCausalLM.from_pretrained("fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")

In [ ]:
inputs = tokenizer("Hello, how are you?", return_tensors="pt")

# Generowanie odpowiedzi
outputs = model.generate(inputs['input_ids'], max_length=50, num_return_sequences=1)

# Dekodowanie wyników
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)